In [1]:
import sys
import os
import shutil
import warnings

In [2]:
import torch
if not torch.cuda.is_available():
    warnings.warn('CUDA is not available.')
else:
    print("CUDA is available 🙂")

import random
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import torch.optim as optim

c:\code\you-look-like\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA is available 🙂


In [3]:
torch.cuda.empty_cache()

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
transformations = transforms.Compose([
    transforms.ToTensor()
])

In [6]:
# class CelebritiesDataset(Dataset):
#     def __init__(self, path='./data/cnn/download_crop/', transform=transformations):
#         super(CelebritiesDataset, self).__init__()
#         self.data = datasets.ImageFolder(path,  transform)    # Create data from folder

#     def __getitem__(self, idx):
#         x, y = self.data[idx]
#         return x, y

#     def __len__(self):
#         return len(self.data)

In [7]:
celebrities_dataset = datasets.ImageFolder("./data/cnn/download_crop/",  transformations)

length_test = int(len(celebrities_dataset) * 0.15)
length_valid = int(len(celebrities_dataset) * 0.15)
length_train = len(celebrities_dataset) - length_test - length_valid

print("total", len(celebrities_dataset))
print(length_train, length_valid, length_test)

train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(celebrities_dataset, [length_train, length_valid, length_test])

total 12023
8417 1803 1803


In [8]:
# The hyperparameters we will use
batch_size = 16
learning_rate = 0.0007

In [9]:
train_dataloader = DataLoader(train_dataset,
								batch_size=batch_size,
								drop_last=True,
								shuffle=True,
								num_workers=4)
valid_dataloader = DataLoader(valid_dataset,
								batch_size=batch_size,
								drop_last=True,
								shuffle=False,
								num_workers=4)
test_dataloader = DataLoader(test_dataset,
								batch_size=batch_size,
								drop_last=True,
								shuffle=False,
								num_workers=4)

In [10]:
import torch.nn as nn
import torch.nn.functional as F

In [11]:
class Celebrities(nn.Module):
    def __init__(self):
        super(Celebrities, self).__init__()

        self.conv1 = nn.Conv2d(3, 100, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(100)
        
        self.conv2 = nn.Conv2d(100, 80, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(80)
        
        self.conv3 = nn.Conv2d(80, 50, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(50)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(32*32*50, 100)
        self.bn4 = nn.BatchNorm1d(100)

        self.fc2 = nn.Linear(100, 80)
        self.bn5 = nn.BatchNorm1d(80)

        self.fc3 = nn.Linear(80, 72)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool(x)

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        # x = x.reshape(x.size(0), 64*64*30)

        x = self.fc1(x)
        x = self.bn4(x)
        # x = F.relu(x)

        x = self.fc2(x)
        x = self.bn5(x)
        # x = F.relu(x)

        x = self.fc3(x)

        return x

In [12]:
model = Celebrities().to(device)
# Print the structure
print(model)

Celebrities(
  (conv1): Conv2d(3, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(100, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(80, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=51200, out_features=100, bias=True)
  (bn4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=100, out_features=80, bias=True)
  (bn5): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=80, out_features=72, bias=True)
)


In [13]:
print("Number of items in dataset: ", len(train_dataloader.dataset))
print("Number of batches: ", len(train_dataloader))

Number of items in dataset:  8417
Number of batches:  526


In [14]:
import cnn
criterion = nn.CrossEntropyLoss()

In [15]:
optimizer = optim.Adam(list(model.parameters()), lr=learning_rate, betas=(0.9, 0.999))

valid_score_best = 0
patience = 2
num_epochs = 300

for e in range(num_epochs):
    print(f"Epoch {e+1}\n-------------------------------")
    train_loss, running_loss = cnn.train_loop(model, train_dataloader, device, optimizer, criterion)
    print('Train: epoch {}: train_loss={:.3f} running_loss={:.3f}'.format(e, train_loss, running_loss))
    train_loss, running_loss = cnn.valid_loop(model, valid_dataloader, device, optimizer, criterion)
    print('Valid: epoch {}: valid_loss={:.3f} running_loss={:.3f}'.format(e, train_loss, running_loss))
    print("\n")
    
    # if valid_score > valid_score_best:
    #     print('Best score: {}. Saving model...'.format(valid_score))
    #     torch.save(model, 'model_params.pt')
    #     valid_score_best = valid_score
    # else:
    #     patience -= 1
    #     print('Score did not improve! {} <= {}. Patience left: {}'.format(valid_score,
    #                                                                       valid_score_best,
    #                                                                       patience))
    # if patience == 0:
    #     print('patience reduced to 0. Training Finished.')
    #     break

Epoch 1
-------------------------------
[    0/  526] train_loss: 0.091, accuracy: 0.000 
[   50/  526] train_loss: 4.181, accuracy: 12.000 
[  100/  526] train_loss: 3.977, accuracy: 19.000 
[  150/  526] train_loss: 3.736, accuracy: 6.000 
[  200/  526] train_loss: 3.596, accuracy: 6.000 
[  250/  526] train_loss: 3.533, accuracy: 19.000 
[  300/  526] train_loss: 3.439, accuracy: 19.000 
[  350/  526] train_loss: 3.340, accuracy: 31.000 
[  400/  526] train_loss: 3.225, accuracy: 25.000 
[  450/  526] train_loss: 3.166, accuracy: 38.000 
[  500/  526] train_loss: 3.048, accuracy: 38.000 
Train: epoch 0: train_loss=1842.520 running_loss=31.000
Valid: epoch 0: valid_loss=337.280 running_loss=6.746


Epoch 2
-------------------------------
[    0/  526] train_loss: 0.053, accuracy: 56.000 
[   50/  526] train_loss: 2.684, accuracy: 12.000 
[  100/  526] train_loss: 2.640, accuracy: 25.000 
[  150/  526] train_loss: 2.632, accuracy: 25.000 
[  200/  526] train_loss: 2.547, accuracy: 31.

KeyboardInterrupt: 